In [12]:
import json
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report, accuracy_score
from pprint import pprint
from string import punctuation
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

In [13]:
class EP:
    
    def __init__(self, ethnicity):  
        
        assert ethnicity in 'iranian polish italian german'.split(), f'incorrect ethnicity: {ethnicity}!'
        
        datafile = f'/Users/ik/Data/names/training-{ethnicity}.csv'
        self.target_col = f'is_{ethnicity}'
        
        self.data = pd.read_csv(datafile)
        
        assert self.target_col in self.data.columns, f'there is no {self.target_col} in data file!'  
        
        try:
            self.NODOUBT_FIRST_NAMES = list({line.strip() for line in open(f'/Users/ik/Data/names/real_{ethnicity}.txt','r').readlines() if line.strip()})
        except:
            self.NODOUBT_FIRST_NAMES = []
        
    def add_nodoubt_names(self):
        
        print(f'adding {len(self.NODOUBT_FIRST_NAMES)} first names..')
        
        self.data = pd.concat([self.data, pd.DataFrame({'full_name': self.NODOUBT_FIRST_NAMES, 
                                                        self.target_col: [1]*len(self.NODOUBT_FIRST_NAMES)})], 
                              ignore_index=True).sample(frac=1.)
        return self

In [14]:
class Selector(BaseEstimator, TransformerMixin):
    """
    select a columns from a data frame and return as a list
    """
    def __init__(self, col_name):
        self.col_name = col_name
    
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        return '_start_' + x[self.col_name] + '_end_'

class WordCount(BaseEstimator, TransformerMixin):
    """
    select a columns from a data frame and return as a list
    """
    def __init__(self):
        pass
    
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        res = x.apply(lambda _: len(_.split())).values.reshape(x.shape[0],1)
        return res

class NameLength(BaseEstimator, TransformerMixin):
    """
    return the length of the full name
    """
    def __init__(self):
        pass
    
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        res = x.str.len().values.reshape(x.shape[0],1)
        return res

class FirstLast(BaseEstimator, TransformerMixin):
    """
    is the first word longer than the last one
    """
    def __init__(self):
        pass
    
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        res = x.apply(lambda _: np.argmax([len(p) for i, p in enumerate(_.split()) if i in [0,len(_.split())-1]])).values.reshape(x.shape[0],1)
        return res

In [15]:
if __name__ == '__main__':
    
    ic = EP('german')
    #.add_nodoubt_names()
    
    # split into the training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(ic.data.drop([ic.target_col], axis=1), 
                                                        ic.data[ic.target_col],  test_size=0.2, 
                                                        random_state=391, stratify=ic.data[ic.target_col])
    

    pipe = Pipeline([('select_fullname', Selector('full_name')),  # df with 1 column
                        ('features', FeatureUnion(
                             [('char_level', CountVectorizer(strip_accents='ascii', analyzer='char', 
                                                          ngram_range=(1, 4))),
                              ('word_level', CountVectorizer(strip_accents='ascii', analyzer='word',
                                                       ngram_range=(1,2))),
                              ('word_count', WordCount()),
                              ('full_name_length', NameLength()),
                              ('firstlast', FirstLast())],
                                 transformer_weights={
                                            'char_level': 0.8,
                                            'word_level': 0.5,
                                            'word_lengths': 1.0,
                                                        })
                     ),
                     ('normalise', Normalizer()),
                     ('pca', TruncatedSVD(n_components=120)),
                    ('clf', SGDClassifier(max_iter=1000))])
    
    param_grid = {'features__transformer_weights': [{'char_level': 0.2,
                                                     'word_level': 0.1,
                                                     'word_count':0.4,
                                                    'full_name_length': 0.5}, 
                                                   {'char_level': 0.9,
                                                     'word_level': 0.2,
                                                     'word_lengths':0.9,
                                                   'full_name_length': 0.3},
                                                   {'char_level': 0.9,
                                                     'word_level': 0.3,
                                                     'word_count':0.2,
                                                   'full_name_length': 0.7}],
                    "clf__loss": ['hinge', 'modified_huber', 'log', 'perceptron'],
                     'clf__penalty': ['l1','l2','elasticnet']}
    
    grid_search = GridSearchCV(pipe, param_grid=param_grid, verbose=10)
    
    grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] clf__loss=hinge, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5} 
[CV]  clf__loss=hinge, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5}, score=0.8388214904679376, total=   4.0s
[CV] clf__loss=hinge, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5} 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.3s remaining:    0.0s


[CV]  clf__loss=hinge, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5}, score=0.8509532062391681, total=   3.8s
[CV] clf__loss=hinge, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5} 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    8.5s remaining:    0.0s


[CV]  clf__loss=hinge, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5}, score=0.8294797687861272, total=   3.8s
[CV] clf__loss=hinge, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3} 


...

[CV]  clf__loss=modified_huber, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3}, score=0.8491329479768787, total=   3.3s
[CV] clf__loss=modified_huber, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.3, 'word_count': 0.2, 'full_name_length': 0.7} 
[CV]  clf__loss=modified_huber, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.3, 'word_count': 0.2, 'full_name_length': 0.7}, score=0.8434430964760254, total=   3.4s
[CV] clf__loss=modified_huber, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.3, 'word_count': 0.2, 'full_name_length': 0.7} 
[CV]  clf__loss=modified_huber, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.3, 'word_count': 0.2, 'full_name_length': 0.7}, score=0.8636626227614096, total=   3.6s
[CV] clf__loss=modified_huber, clf__penalty=l1, features__transfo

[CV]  clf__loss=log, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5}, score=0.8203350664355864, total=   5.2s
[CV] clf__loss=log, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5} 
[CV]  clf__loss=log, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5}, score=0.8109826589595376, total=   5.3s
[CV] clf__loss=log, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3} 
[CV]  clf__loss=log, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3}, score=0.854419410745234, total=   4.5s
[CV] clf__loss=log, clf__penalty=l1, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths

[CV]  clf__loss=log, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.3, 'word_count': 0.2, 'full_name_length': 0.7}, score=0.853264009243212, total=   3.9s
[CV] clf__loss=log, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.3, 'word_count': 0.2, 'full_name_length': 0.7} 
[CV]  clf__loss=log, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.3, 'word_count': 0.2, 'full_name_length': 0.7}, score=0.8317919075144509, total=   4.0s
[CV] clf__loss=perceptron, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5} 
[CV]  clf__loss=perceptron, clf__penalty=l1, features__transformer_weights={'char_level': 0.2, 'word_level': 0.1, 'word_count': 0.4, 'full_name_length': 0.5}, score=0.6961294049682265, total=   3.9s
[CV] clf__loss=perceptron, clf__penalty=l1, features__transformer_weights={'char_leve

[CV]  clf__loss=perceptron, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3}, score=0.8174465626805315, total=   3.3s
[CV] clf__loss=perceptron, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3} 
[CV]  clf__loss=perceptron, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3}, score=0.8711727325245523, total=   3.5s
[CV] clf__loss=perceptron, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3} 
[CV]  clf__loss=perceptron, clf__penalty=elasticnet, features__transformer_weights={'char_level': 0.9, 'word_level': 0.2, 'word_lengths': 0.9, 'full_name_length': 0.3}, score=0.8173410404624277, total=   3.6s
[CV] clf__loss=perceptron, clf__penal

[Parallel(n_jobs=1)]: Done 108 out of 108 | elapsed:  6.9min finished


In [16]:
print(classification_report(y_test, grid_search.predict(X_test)))

             precision    recall  f1-score   support

          0       0.89      0.86      0.88       709
          1       0.84      0.88      0.86       589

avg / total       0.87      0.87      0.87      1298



In [17]:
test_data = pd.DataFrame({'full_name': ['mohammad kumar', 'chan','stanislaw', 'mohammad', 'mehrdad','myriam',
                                         'nejad myriam','hosein', 'xi','babak', 'igor', 
                                        'andrzej woods', 'kumar', 'otto floms', 'ahmad lopes zeto', 'bob', 'mario razzi', 'john reed']})

test_data['prediction'] = grid_search.predict(test_data)
test_data['prediction'] = test_data['prediction'].apply(lambda _: 'yes' if _ else 'no')

In [18]:
test_data

,full_name,prediction
0,mohammad kumar,no
1,chan,no
2,stanislaw,no
3,mohammad,no
4,mehrdad,no
5,myriam,no
6,nejad myriam,no
7,hosein,yes
8,xi,no
9,babak,no


In [19]:
grid_search.best_params_

{'clf__loss': 'modified_huber',
 'clf__penalty': 'elasticnet',
 'features__transformer_weights': {'char_level': 0.9,
  'full_name_length': 0.7,
  'word_count': 0.2,
  'word_level': 0.3}}